# Analysis of the beamtime data

## Imports

In [1]:
from google.colab import drive
drive.mount('/content/drive')

filepath_drive = "/content/drive/.shortcut-targets-by-id/1B48ps8379Krem2Eym3IJNaUK-_pZ9Q2w/NP2012-AVF72-05"

Mounted at /content/drive


In [2]:
cd /content/drive/MyDrive/data-engineering-utokyo/notebooks

/content/drive/MyDrive/data-engineering-utokyo/notebooks


In [4]:
import sys
sys.path.insert(0,'..')

from datetime import datetime

from src.constants.mot_constants import c_ccd, c_cmos_Fr_20220918, c_cmos_Rb_20220918
from src.recorders.ssd_recorder import SSDRecorder
from src.recorders.file_recorder import FileRecorder
from src.recorders.image_file_recorder import ImageFileRecorder
from src.analyses.analysis import ResultParameter
from src.analyses.ssd_analysis import SSDAnalysis
from src.analyses.image_analysis import ImageAnalysis
from src.analyses.mkdir import create_folders
from src.analyses.runner import Runner
from src.analyses.fit_mot_number import MOTMLE

## Preparation with Rubidium (Sunday, 9:00 - 13:00)
During this time, no images were taken. Here, we can run the SSD analysis if desired. 

## First measurements with Francium (Sunday, 13:00 - 21:00)
During this time, we took both camera images and the signal from the current integrator. We first perform just the analysis of the camera images. 



In [8]:
# Input 
image_folder = f"{filepath_drive}/mot_data/"
match = ".*20220918.*cmos_roidata.*cmos.*.csv$" 
c = c_cmos_Fr_20220918
min_signal = 0
time_interval = (
    datetime(2022, 9, 18, 13, 0, 0), 
    datetime(2022, 9, 18, 21, 0, 0)
    )
use_n_reference_images = 500
dead_pixel_percentile = 2.0 # [%], must between 0 and 100

# Output 
run_name = f"Sunday_Fr_1300_to_2100_min_signal{min_signal}_nref{use_n_reference_images}_deadp{str(int(dead_pixel_percentile))}"
plot_path = f"{filepath_drive}/plots/{run_name}/" 
result_path = f"{filepath_drive}/results/{run_name}/" 

In [11]:
# Make dirs
create_folders(plot_path, result_path)

# Setup result parameters
result_param_image = ResultParameter(
    image_src=plot_path+"image/",
    image_extension=".png",
    result_filepath=result_path+"image_analysis_results.csv"
    )

# Setup recorders
image_recorder = ImageFileRecorder(
    filepath=image_folder,
    match=match 
    )

# Setup analyses
reference_image_filepaths = image_recorder.get_table().head(use_n_reference_images)["filepath"]

['/content/drive/.shortcut-targets-by-id/1B48ps8379Krem2Eym3IJNaUK-_pZ9Q2w/NP2012-AVF72-05/mot_data/20220918-122539_data/cmos_roidata/0918122554/cmos_000257.csv', '/content/drive/.shortcut-targets-by-id/1B48ps8379Krem2Eym3IJNaUK-_pZ9Q2w/NP2012-AVF72-05/mot_data/20220918-122539_data/cmos_roidata/0918122554/cmos_000259.csv', '/content/drive/.shortcut-targets-by-id/1B48ps8379Krem2Eym3IJNaUK-_pZ9Q2w/NP2012-AVF72-05/mot_data/20220918-122539_data/cmos_roidata/0918122554/cmos_000260.csv', '/content/drive/.shortcut-targets-by-id/1B48ps8379Krem2Eym3IJNaUK-_pZ9Q2w/NP2012-AVF72-05/mot_data/20220918-122539_data/cmos_roidata/0918122554/cmos_000262.csv', '/content/drive/.shortcut-targets-by-id/1B48ps8379Krem2Eym3IJNaUK-_pZ9Q2w/NP2012-AVF72-05/mot_data/20220918-122539_data/cmos_roidata/0918122554/cmos_000264.csv', '/content/drive/.shortcut-targets-by-id/1B48ps8379Krem2Eym3IJNaUK-_pZ9Q2w/NP2012-AVF72-05/mot_data/20220918-122539_data/cmos_roidata/0918122554/cmos_000265.csv', '/content/drive/.shortcut-t

In [12]:
perform_analysis = MOTMLE(c=c, 
                          references=reference_image_filepaths,
                          do_subtract_dead_pixels=True,
                          dead_pixel_percentile=dead_pixel_percentile).perform_analysis
image_analysis = ImageAnalysis(
    recorder=image_recorder,
    perform_analysis=perform_analysis, 
    result_param=result_param_image,
    min_signal=min_signal,
    time_interval=time_interval
    )
    
# Setup runner
runner = Runner(analyses=[image_analysis])
runner.run(cycles=1, period_s=5)


Output hidden; open in https://colab.research.google.com to view.

## Second measurement with Francium (Sunday, 9:00 - 12:00)

## Third measurement with Francium (Sunday, 12:00 - 18:00)
